### Unsloth

Load up `DeepSeek-R1-Distill-Qwen-7B`, and set parameters

In [2]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.50.0. vLLM: 0.7.2.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 1. Max memory: 47.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = True]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/deepseek-r1-distill-qwen-7b-unsloth-bnb-4bit with actual GPU utilization = 24.31%
Unsloth: Your GPU has CUDA compute capability 8.6 with VRAM = 47.32 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 5.38 GB. Also swap space = 6 GB.
INFO 04-02 10:42:40 config.py:542] This model supports multiple tasks: {'embed', 'generate', 'score', 'classify', 'reward'}. Defaulting to 'generate'.
Unsloth: vLLM Bitsandbytes

[W402 10:42:44.031267853 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 04-02 10:42:44 loader.py:1102] Loading weights with BitsAndBytes quantization.  May take a while ...
INFO 04-02 10:42:45 weight_utils.py:252] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.52s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:03<00:00,  1.67s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:03<00:00,  1.65s/it]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.48s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:03<00:00,  1.80s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:03<00:00,  1.75s/it]



INFO 04-02 10:44:07 model_runner.py:1115] Loading model weights took 8.0151 GB
INFO 04-02 10:44:07 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 04-02 10:44:10 worker.py:267] Memory profiling takes 2.52 seconds
INFO 04-02 10:44:10 worker.py:267] the current vLLM instance can use total_gpu_memory (47.32GiB) x gpu_memory_utilization (0.24) = 11.50GiB
INFO 04-02 10:44:10 worker.py:267] model weights take 8.02GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 1.05GiB; the rest of the memory reserved for KV Cache is 2.38GiB.
INFO 04-02 10:44:10 executor_base.py:110] # CUDA blocks: 2785, # CPU blocks: 7021
INFO 04-02 10:44:10 executor_base.py:115] Maximum concurrency for 1024 tokens per request: 43.52x
INFO 04-02 10:44:14 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error oc

Capturing CUDA graph shapes: 100%|██████████| 27/27 [00:26<00:00,  1.02it/s]

INFO 04-02 10:44:40 model_runner.py:1562] Graph capturing finished in 26 secs, took 0.59 GiB
INFO 04-02 10:44:40 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 32.96 seconds



Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [3]:
import re
from datasets import load_dataset, Dataset

# Load and prep dataset
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

def extract_hash_answer(text: str) -> str | None:
    if "####" not in text:
        return None
    return text.split("####")[1].strip()

# uncomment middle messages for 1-shot prompting
def get_gsm8k_questions(split = "train") -> Dataset:
    data = load_dataset('openai/gsm8k', 'main')[split] # type: ignore
    data = data.map(lambda x: { # type: ignore
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': extract_hash_answer(x['answer'])
    }) # type: ignore
    return data # type: ignore

dataset = get_gsm8k_questions()

# Reward functions
def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    q = prompts[0][-1]['content']
    extracted_responses = [extract_xml_answer(r) for r in responses]
    print('-'*20, f"Question:\n{q}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}")
    return [2.0 if r == a else 0.0 for r, a in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def strict_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def soft_format_reward_func(completions, **kwargs) -> list[float]:
    """Reward function that checks if the completion has a specific format."""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    matches = [re.match(pattern, r) for r in responses]
    return [0.5 if match else 0.0 for match in matches]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [4]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 256,
    max_completion_length = 200,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 250,
    save_steps = 250,
    max_grad_norm = 0.1,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8


And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [5]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        xmlcount_reward_func,
        soft_format_reward_func,
        strict_format_reward_func,
        int_reward_func,
        correctness_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1 | Total steps = 250
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 161,480,704/7,000,000,000 (2.31% trained)


-------------------- Question:
A concert ticket costs $40. Mr. Benson bought 12 tickets and received a 5% discount for every ticket bought that exceeds 10. How much did Mr. Benson pay in all? 
Answer:
476 
Response:
First, I need to determine how many concert tickets Mr. Benson bought and identify how many of those qualify for a discount. He bought 12 tickets in total, and the promotion offers a 5% discount for every ticket exceeding 10. Therefore, the number of tickets with a discount is 12 minus 10, which equals 2 tickets.

Next, I'll calculate the discounted price for these 2 tickets. The original price per ticket is $40, so the discount amount is 5% of $40, which is $2. This means each discounted ticket costs $38.

Now, I'll calculate the total cost for the 2 discounted tickets by multiplying the discounted price by 2. That gives 2 multiplied by $38, totaling $76.

For the remaining 10 tickets, there's no discount, so each ticket costs the full price of $40. The total cost for thes

Step,Training Loss,reward,reward_std,completion_length,kl,rewards / xmlcount_reward_func,rewards / soft_format_reward_func,rewards / strict_format_reward_func,rewards / int_reward_func,rewards / correctness_reward_func
1,-0.000000,0.046875,0.064694,200.000000,0.000000,0.046875,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,200.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.015625,0.044194,200.000000,0.000331,0.015625,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.046875,0.064694,200.000000,0.000267,0.046875,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.031250,0.057864,200.000000,0.000230,0.031250,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,200.000000,0.000230,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,200.000000,0.000255,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.046875,0.064694,200.000000,0.000379,0.046875,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.046875,0.064694,200.000000,0.000278,0.046875,0.000000,0.000000,0.000000,0.000000
10,0.000000,0.015625,0.044194,200.000000,0.000215,0.015625,0.000000,0.000000,0.000000,0.000000


-------------------- Question:
Jane is trying to decide whether to buy a house or a trailer. A house costs $480,000 and a trailer costs $120,000. Each loan will be paid in monthly installments over 20 years. How much more is the monthly payment on the house compared to the trailer? 
Answer:
1500 
Response:
Jane wants to buy either a house or a trailer, each with a 20-year loan. The house costs $480,000, and the trailer costs $120,000. To find the difference in monthly payments, I need to calculate the monthly payment for each using the loan formula. 

First, I'll set up the loan parameters: principal amounts are $480,000 for the house and $120,000 for the trailer. The loan term is 20 years, which is 240 months. I'll assume an interest rate, maybe 5% for both loans. 

Using the formula for monthly payments on a fixed-rate loan, I'll plug in the numbers for both the house and the trailer. The monthly payment for the house will be higher because the principal is larger. After calculating 

TrainOutput(global_step=250, training_loss=0.00016238416443639637, metrics={'train_runtime': 8414.7879, 'train_samples_per_second': 0.238, 'train_steps_per_second': 0.03, 'total_flos': 0.0, 'train_loss': 0.00016238416443639637})

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [6]:
text = tokenizer.apply_chat_template([
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:51<00:00, 51.48s/it, est. speed input: 0.27 toks/s, output: 17.50 toks/s]


'Okay, so I need to figure out how many times the letter \'r\' appears in the word "strawberry." Let me break this down step by step because I want to make sure I don\'t miss any letters, especially since I sometimes get confused with similar-sounding letters.\n\nFirst, I\'ll write out the word: strawberry. Now, I\'ll go through each letter one by one and count whenever I see an \'r\'. \n\nStarting from the beginning: \'s\' is the first letter. That\'s not an \'r\', so I move on. The next letter is \'t\'. Also not an \'r\'. Then comes \'r\' again. Okay, that\'s one \'r\'. I\'ll mark that down as 1.\n\nContinuing, after \'r\' comes \'a\'. Not an \'r\'. Next is \'w\'. Still not an \'r\'. Then \'b\'. Not \'r\' either. After that, there\'s another \'r\'. That\'s my second \'r\', so my count is now 2.\n\nI\'ll keep going. The next letters are \'a\' and \'y\', which aren\'t \'r\'s. Then there\'s another \'r\'. That brings my total to 3. Finally, the last letters are \'r\' and \'e\'. So that\

And now with the LoRA we just trained with GRPO - we first save the LoRA first!

In [7]:
model.save_lora("grpo_saved_lora")

Now we load the LoRA and test:

In [8]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "How many r's are in strawberry?"},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:16<00:00, 16.08s/it, est. speed input: 2.30 toks/s, output: 15.17 toks/s]


'I need to determine how many times the letter \'r\' appears in the word "strawberry."\n\nFirst, I\'ll look at each letter in the word individually.\n\nI\'ll start with \'s\' and count that as one.\n\nNext is \'t\', which isn\'t an \'r,\' so the count remains one.\n\nThen there\'s \'r,\' so that\'s my second \'r.\'\n\nAnother \'a\' doesn\'t add to the count.\n\nFollowing that, there\'s another \'w,\' which isn\'t an \'r.\'\n\nAnother \'a\' doesn\'t change the count either.\n\nNow, I come across the third \'r,\' which brings the total to three.\n\nFinally, there\'s \'b\' and \'e,\' but those aren\'t \'r\'s either.\n\nAfter reviewing all the letters, I conclude that there are three \'r\'s in "strawberry."\n</think>\n\n<reasoning>\nThe word "strawberry" contains the following letters: s, t, r, a, w, b, e, r, r. Counting the occurrences of the letter \'r\', we find it appears three times.\n</reasoning>\n\n<answer>\n\\boxed{3}\n</answer>'

In [10]:
text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : "How many e's are in dependence? Calculate it."},
], tokenize = False, add_generation_prompt = True)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.8,
    top_p = 0.95,
    max_tokens = 1024,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Processed prompts: 100%|██████████| 1/1 [00:13<00:00, 13.51s/it, est. speed input: 2.96 toks/s, output: 19.62 toks/s]


'To determine how many \'e\'s are in the word "dependence," I first need to examine each letter individually.\n\nI\'ll start by spelling out the word and labeling each letter with its position to keep track.\n\nD - 1\ne - 2\np - 3\ne - 4\nn - 5\nd - 6\ne - 7\nn - 8\ng - 9\n\nAs I go through each letter, I\'ll count each occurrence of the letter \'e.\' From this analysis, I can see that the letters at positions 2, 4, and 7 are \'e\'s.\n\nAdding these up, there are three \'e\'s in the word "dependence."\n</think>\n\n<reasoning>\nThe word "dependence" is spelled as D - E - P - E - N - D - E - N - G.\n\nCounting the number of \'e\'s:\n1. **First \'e\'**: Position 2\n2. **Second \'e\'**: Position 4\n3. **Third \'e\'**: Position 7\n\nThere are **3 \'e\'s** in the word "dependence."\n</reasoning>\n\n<answer>\nThe number of \'e\'s in "dependence" is \\boxed{3}.\n</answer>'

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [9]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [11]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
